In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In this notebook we investigate some properties of matrices that approximate the second-derivative operator via a 3-point centered difference:

$$
  u''(x_j) \approx \frac{U_{j+1}-2U_j + U_{j-1}}{h^2}.
$$

# Properties of the columns of $A^{-1}$

The following function constructs the matrix with the boundary values included as unknowns:

In [ ]:
def A_matrix_full(m):
    em = np.ones(m+2)
    e1=np.ones(m+1)
    A = (np.diag(-2*em,0)+np.diag(e1,-1)+np.diag(e1,1))*(m+1)**2
    A[0,0] = 1.; A[0,1]=0.; A[-1,-1]=1.; A[-1,-2]=0.
    return A

Here we build the matrix with $h=1/10$ and check its structure:

In [ ]:
m=9
h = 1./(m+1)
x = np.linspace(0,1,m+2)
A=A_matrix_full(m)
print(A)

As discussed in class (and in Section 2.11 of LeVeque), the columns of the inverse of $A$ are discrete versions of the Green's function for this problem.  Here we plot one of them:

In [ ]:
B = np.linalg.inv(A)
plt.plot(x,B[:,3]/h);

This is the exact solution of $u''(x)=f(x)$ when $f(x)=\delta(x-0.3)$.

Next, we plot all the interior columns of $B$, which are the Green's functions corresponding to $f(x)$ being a $\delta$-function located at each of the grid points.

In [ ]:
for j in range(1,m+1):
    plt.plot(x,B[:,j]/h);

Finally, the first and last columns of $B$ correspond to the exact solution of the problem when $f(x)=0$ and just one of the boundary values is non-zero.

In [ ]:
plt.plot(x,B[:,0])

In [ ]:
plt.plot(x,B[:,-1])

# Eigenvalues and eigenvectors of $A$

Now we examine the spectral properties of the matrix $A$, as discussed in Section 2.10 of LeVeque.  This time it's convenient to eliminate the values of $u$ at the boundaries, by using the boundary conditions.

In [ ]:
def A_matrix(m):
    em = np.ones(m)
    e1=np.ones(m-1)
    return (np.diag(-2*em,0)+np.diag(e1,-1)+np.diag(e1,1))*(m+1)**2

We proved that the spectral radius of $A^{-1}$ is bounded as $h\to 0$, and in fact that it should approach $1/\pi^2$.  Here we test this computationally.

In [ ]:
for m in(9,99,999):
    B = np.linalg.inv(A_matrix(m))
    print(np.max(np.abs(np.linalg.eigvals(B))))

In [ ]:
print(1./np.pi**2)

We see that, indeed, the spectral radius is converging to $1/\pi^2$.

Finally, we look in some detail at the eigenvectors of $A$.  Since $A$ is a discrete version of the differential operator $d^2/dx^2$, the eigenvectors of $A$ might be expected to approximate the eigenfunctions of this operator.

In [ ]:
m=50
A=A_matrix(m)
[lamda,u]=np.linalg.eig(A)
ind = np.argsort(abs(lamda))
spectrum = lamda[ind]
print('Eigenvalues: ', spectrum)
B = np.linalg.inv(A)
print('Norm of A^{-1}:', np.linalg.norm(B,2))

# sort the eigenvectors too
v=np.zeros(np.shape(u))
for i in range(len(ind)):
    v[:,i]=u[:,ind[i]]

In [ ]:
from ipywidgets import interact, widgets

def plot_eigenvector(p):
    x=np.linspace(0,1,m+2)
    plt.clf()
    plt.plot(x[1:-1],v[:,p])
    plt.title('Eigenvector $u^{'+str(p)+'}$, corresponding to eigenvalue $\lambda^{'+str(p)+'}='+str(spectrum[p])+'$')

interact(plot_eigenvector, p=widgets.IntSlider(min=0,max=m-1,value=0));

The interactive illustration above allows you to examine the eigenvectors corresponding to different eigenvalues of $A$.  Notice that each eigenvector looks like a sinusoidal function; in fact, the entries of each eigenvector are just the values of a certain sine function, sample at the grid points.  This is also discussed in section 2.10 of LeVeque.